In [23]:
import pandas as pd
from sqlalchemy import create_engine
from pgadmin_keys import username
from pgadmin_keys import password

# Extract

The first file we are going to be using is a list of breweries in the US. This csv has columns for name, address, website, establishment type, state, and number of breweries in that state.
The second file we are going to be using is also a csv of state population data from the US census. This file also contains columns giving the latitude and longitude of the given state's capital. Both of these datasets were available on Kaggle.com.

In [2]:
pop_file = "Resources/2019_Census_US_Population_Data_By_State_Lat_Long.csv"
pop_df = pd.read_csv(pop_file)
pop_df.head()

,STATE,POPESTIMATE2019,lat,long
0,Alabama,4903185,32.377716,-86.300568
1,Alaska,731545,58.301598,-134.420212
2,Arizona,7278717,33.448143,-112.096962
3,Arkansas,3017804,34.746613,-92.288986
4,California,39512223,38.576668,-121.493629


In [3]:
print(f'There are {pop_df["STATE"].count()} rows in the population dataset.')

There are 51 rows in the population dataset.


In [4]:
brew_file = "Resources/breweries_us 2.csv"
brew_df = pd.read_csv(brew_file)
brew_df.head()

,brewery_name,type,address,website,state,state_breweries
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,california,284
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",http://www.valleybrew.com/,california,284
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",http://www.valleybrew.com/,california,284
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",http://www.ukiahbrewingco.com/,california,284
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",http://www.tustinbrewery.com/,california,284


In [5]:
print(f'There are {brew_df["state"].nunique()} unique states in the brewery dataset.')

There are 51 unique states in the brewery dataset.


# Transform

For the population data set, we are going to copy only the state and population columns, essentially dropping the latitude and longitude columns. Next we are going to drop all rows related to the District of Columbia, since the breweries dataset does not have any breweries in the District of Columbia. We are also going to rename the columns to be consistent with the table we are going to make in SQL.

In [6]:
# Create a filtered dataframe from specific columns
pop_cols = ["STATE", "POPESTIMATE2019"]
pop_transformed= pop_df[pop_cols].copy()
pop_transformed.head()

,STATE,POPESTIMATE2019
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223


In [7]:
# Rename the column headers
pop_transformed = pop_transformed.rename(columns={"STATE": "state",
                                                          "POPESTIMATE2019": "population"
                                                          })
pop_transformed.head(20)

,state,population
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223
5,Colorado,5758736
6,Connecticut,3565287
7,Delaware,973764
8,District of Columbia,705749
9,Florida,21477737


In [8]:
#drop District of Columbia
pop_transformed = pop_transformed[pop_transformed.state != "District of Columbia"]
pop_transformed.head(20)

,state,population
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223
5,Colorado,5758736
6,Connecticut,3565287
7,Delaware,973764
9,Florida,21477737
10,Georgia,10617423


In [9]:
#Reset Indices
pop_transformed=pop_transformed.reset_index(drop=True)
pop_transformed.head(20)

,state,population
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223
5,Colorado,5758736
6,Connecticut,3565287
7,Delaware,973764
8,Florida,21477737
9,Georgia,10617423


In [10]:
# Copy the dataframe
brew_cols = ["brewery_name", "type", "address", "website", "state", "state_breweries"]
brew_transformed= brew_df[brew_cols].copy()
brew_transformed.head()

,brewery_name,type,address,website,state,state_breweries
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,california,284
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",http://www.valleybrew.com/,california,284
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",http://www.valleybrew.com/,california,284
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",http://www.ukiahbrewingco.com/,california,284
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",http://www.tustinbrewery.com/,california,284


In [12]:
# Rename the column headers
brew_transformed = brew_transformed.rename(columns={"type": "brewery_type",
                                                    "state": "brewery_state",
                                                    "state_breweries": "brewery_state_count",
                                                    "address": "brewery_address",
                                                    "website": "brewery_website"
                                                          })
brew_transformed.head(20)

,brewery_name,brewery_type,brewery_address,brewery_website,brewery_state,brewery_state_count
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,california,284
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",http://www.valleybrew.com/,california,284
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",http://www.valleybrew.com/,california,284
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",http://www.ukiahbrewingco.com/,california,284
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",http://www.tustinbrewery.com/,california,284
5,Trumer Brauerei,Microbrewery,"1404 4th St., Berkeley, California, 94608",http://www.trumer-international.com/,california,284
6,Trumer Brauerei,RegionalBrewery,"1404 Fourth St., Berkeley, California, 94710",http://www.trumer-international.com/,california,284
7,Triple Rock Brewing Co.,Brewpub,"1920 Shattuck Ave, Berkeley, California, 94704",http://www.triplerock.com/,california,284
8,Tied House Cafe & Brewery (San Jose),Brewpub,"65 N. San Pedro, San Jose, California, 95110",http://www.tiedhouse.com/,california,284
9,Tied House Cafe & Brewery (Mt. View),Brewpub,"954 Villa St., Mountain View, California, 94041",http://www.tiedhouse.com/,california,284


In [14]:
#drop Puerto Rico
brew_transformed = brew_transformed[brew_transformed.brewery_state != "puerto-rico"]
brew_transformed.head(20)

,brewery_name,brewery_type,brewery_address,brewery_website,brewery_state,brewery_state_count
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,california,284
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",http://www.valleybrew.com/,california,284
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",http://www.valleybrew.com/,california,284
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",http://www.ukiahbrewingco.com/,california,284
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",http://www.tustinbrewery.com/,california,284
5,Trumer Brauerei,Microbrewery,"1404 4th St., Berkeley, California, 94608",http://www.trumer-international.com/,california,284
6,Trumer Brauerei,RegionalBrewery,"1404 Fourth St., Berkeley, California, 94710",http://www.trumer-international.com/,california,284
7,Triple Rock Brewing Co.,Brewpub,"1920 Shattuck Ave, Berkeley, California, 94704",http://www.triplerock.com/,california,284
8,Tied House Cafe & Brewery (San Jose),Brewpub,"65 N. San Pedro, San Jose, California, 95110",http://www.tiedhouse.com/,california,284
9,Tied House Cafe & Brewery (Mt. View),Brewpub,"954 Villa St., Mountain View, California, 94041",http://www.tiedhouse.com/,california,284


In [15]:
print(f'There are {brew_transformed["brewery_state"].nunique()} unique states in the brewery dataset.')

There are 50 unique states in the brewery dataset.


In [19]:
#Add id column for data table primary key
new_column=brew_transformed.index
brew_transformed["id"]=new_column
brew_transformed.head()

/Users/nathanwitte/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,brewery_name,brewery_type,brewery_address,brewery_website,brewery_state,brewery_state_count,id
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,california,284,0
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",http://www.valleybrew.com/,california,284,1
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",http://www.valleybrew.com/,california,284,2
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",http://www.ukiahbrewingco.com/,california,284,3
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",http://www.tustinbrewery.com/,california,284,4


In [20]:
# Set index to be id
brew_transformed.set_index("id", inplace=True)

brew_transformed.head()

,brewery_name,brewery_type,brewery_address,brewery_website,brewery_state,brewery_state_count
id,,,,,,
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,california,284
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",http://www.valleybrew.com/,california,284
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",http://www.valleybrew.com/,california,284
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",http://www.ukiahbrewingco.com/,california,284
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",http://www.tustinbrewery.com/,california,284


# Load

We first created the database "breweries_db" in pgAdmin. We next created two tables in that database, one called "population" and another called "breweries." Each table has the corresponding columns to the two DataFrames in this Jupyter Notebook. 

In [24]:
#Create connection to PGAdmin
connection_string = f"{username}:{password}@localhost:5432/breweries_db"
engine = create_engine(f'postgresql://{connection_string}') # Confirm tables
engine.table_names()

['population', 'breweries']

In [27]:
#Load population Data Frame to SQL table in PGAdmin
pop_transformed.to_sql(name='population', con=engine, if_exists='append', index=False)

In [28]:
#Load brewery Data Frame to SQL table in PGAdmin
brew_transformed.to_sql(name='breweries', con=engine, if_exists='append', index=True)